In [71]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from fuzzywuzzy import process
import spacy

In [2]:
inp_data = pd.read_json("intents.json")

In [3]:
inp_data.head()

,intent_model
0,"{'context': 'appointment', 'utterances': ['I n..."
1,"{'context': 'insurance', 'utterances': ['Need ..."
2,"{'context': 'lab', 'utterances': ['Need to und..."


In [8]:
#from pandas.io.json import json_normalize
df = pd.json_normalize(inp_data["intent_model"])

In [9]:
df

,context,utterances
0,appointment,"[I need help with doctor appointment, I am loo..."
1,insurance,"[Need help with insurance claim, What are the ..."
2,lab,"[Need to undergo diagnosis, How much is the la..."


In [96]:
def getContext(match_string):
    for idx, i in enumerate(df['utterances']):
        if match_string in i:
            return df['context'][idx]
    return None

In [97]:
print(getContext('Need to undergo diagnosis'))

lab


In [12]:
flatten_utterances_corpus = [val for sublist in df['utterances'] for val in sublist]
flatten_utterances_corpus

['I need help with doctor appointment',
 'I am looking for appointment',
 'Is the doctor available today',
 'Need help with insurance claim',
 'What are the insurance tie ups available',
 'I need claim settlement',
 'Need to undergo diagnosis',
 'How much is the lab test',
 'When can I get my lab report']

In [13]:
supported_diseases = ['fever', 'cardiac', 'inflammation', 'throat pain', 'heart burn', 'head ache']

In [20]:
detected_disease = process.extract('I am suffering from fever', supported_diseases)[0]
detected_disease

('fever', 90)

In [21]:
def detected_disease(input_query):
    det = process.extract(input_query, supported_diseases)
    if(det[0][1] > 50):
        return det[0][0]
    else:
        return None

In [25]:
print(detected_disease("I am suffering from fever"))

fever


In [28]:
nlp = spacy.load('en_core_web_sm')

In [37]:
def get_time(doc):
    if doc.ents:
        for ent in doc.ents:
            if(ent.label_ == 'DATE'):
                duration = ent.text
                return duration
            return None
    else:
        return None

In [39]:
print(get_time(nlp("suffering from 3 days")))

3 days


In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
stopwords_list = stopwords.words('english')

In [72]:
#vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords_list)
vectorizer = TfidfVectorizer(max_features=3000, stop_words=stopwords_list)

In [73]:
X = vectorizer.fit_transform(flatten_utterances_corpus)

In [74]:
count_dense = X.todense()

In [75]:
cosine_similarity(count_dense[7,:], count_dense[8,:])[0][0]

0.2629108679190545

In [83]:
test_query = vectorizer.transform(["Need to undergo diagnosis"])

In [88]:
cosine_similarity(test_query, count_dense[6,:])[0][0]

1.0

In [92]:
def getContext_details(input_query):
    input_query_vector = vectorizer.transform([input_query])
    max_score = 0
    best_match = None
    context = None
    for ind, vector in enumerate(count_dense):
        sim_score = cosine_similarity(input_query_vector, count_dense[ind,:])[0][0]
        if sim_score>max_score:
            max_score = sim_score
            best_match = flatten_utterances_corpus[ind]
            context = getContext(best_match)
    return max_score, best_match, context
    

In [102]:
print(getContext_details("I want to settle claim"))

(0.5781259620072153, 'I need claim settlement', 'insurance')


In [103]:
def show_ents(doc):
    global name
    global age
    if doc.ents:
        for ent in doc.ents:
            if(ent.label_ == 'PERSON'):
                name = ent.text
            if(ent.label_ == 'DATE'):
                age = ent.text
        return True
    else:
        return False

In [115]:
name = None
age = None
flag = True
date = None
detected_dis = None
symptoms_duration = None
context = None

In [116]:
while flag:
    query = input("User > ")
    if(query =='exit'):
         flag=False
    elif name:
        if(context):
            if(detected_dis):
                if(symptoms_duration):
                    if(date):
                        print('Here is the summary \n Name and age : ',name,age,'\n Symptoms: ', detected_dis, '\n Symptoms Duration: ',symptoms_duration, '\n Appointment Date: ',date)
                        print('c ya soon!')
                        flag=False 
                    else:
                        date = get_time(nlp(query))
                        if(date):
                            print("bot > :) ", 'Noted, your appointment is confirmed for the day', date)
                        else:
                            print("bot > :) ", 'Thank you for the information, Can you tell me what date you are looking for appointment')                
                        
                else:
                    symptoms_duration = get_time(nlp(query))
                    if(symptoms_duration):
                        print("bot > :) ", 'Thank you for the information, Can you tell me what date you are looking for appointment')
                    else:
                        print("bot > :) ", 'Okay, could you please tell me from how long you are observing the symptoms')
                                                             
            else:
                detected_dis = detected_disease(query)
                if(detected_dis):
                    print("bot > :) ", 'Okay, could you please tell me from how long you are observing the symptoms')
                else:
                    print("bot > :) ", 'Can you tell me more about your symptoms')
                    
                
        else:
            _,_,context = getContext_details(query)
            if(context):
                print("bot > :) ", 'Sure, Can you tell me more about your symptoms')
            else:
                print("bot > :) ", 'I am not sure, how can I help you with your ask. I am still learning and may be next time I will be able to answer you.')
                     
            
    else:
        if(show_ents(nlp(query))):
            print("bot > :) ", 'Hi,' ,name,' How can I help you today' )
        else:
            print("bot > :) ", 'Hi, Please tell me your name and age' )

User >  hi


bot > :)  Hi, Please tell me your name and age


User >  I am Priya and I am 34 years old


bot > :)  Hi, Priya  How can I help you today


User >  I am looking for doctor appointment


...

bot > :)  Thank you for the information, Can you tell me what date you are looking for appointment


User >  I need appointment for 15-07-2020


bot > :)  Noted, your appointment is confirmed for the day 15-07-2020


User >  Thank You


Here is the summary 
 Name and age :  Priya 34 years old 
 Symptoms:  fever 
 Symptoms Duration:  around 3 days 
 Appointment Date:  15-07-2020
c ya soon!
